<a href="https://colab.research.google.com/github/maleehamcgill/Customer-Reviews-Insights-Pipeline/blob/etl/save_to_db_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from app.database import SessionLocal
from app.models import Review, ReviewAnalysis
from fetch_reviews import fetch_reviews
from process_reviews import analyze_review

def save():
    db = SessionLocal()
    reviews = fetch_reviews()
    for r in reviews:
        review = Review(
            source="Amazon",
            product_id="EXAMPLE",
            review_text=r["text"],
            rating=int(float(r["rating"].split()[0])),
            review_date=r["date"]
        )
        db.add(review)
        db.commit()
        db.refresh(review)

        analysis = analyze_review(r["text"])
        sentiment, topics = analysis.split("\n")
        sentiment = sentiment.split(":")[-1].strip()
        topics = topics.split(":")[-1].strip()

        review_analysis = ReviewAnalysis(
            review_id=review.id,
            sentiment=sentiment,
            topics=topics
        )
        db.add(review_analysis)
        db.commit()
    db.close()

if __name__ == "__main__":
    save()
